In [1]:
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint

In [2]:
## Path for files downloaded from Samsung Health
data_dir  = "/persistentStorage/SHealth_DataSet/Shealth/"
json_path = "/persistentStorage/SHealth_DataSet/Shealth/jsons/"

sleep_file_prefix      = "com.samsung.shealth.sleep"
sleep_data_file_prefix = "com.samsung.shealth.sleep_data"

sleep_file = None
sleep_data_file = None

for file in os.listdir(data_dir):
    if ((sleep_file_prefix in file) and
        (sleep_data_file_prefix not in file) and
        (file.endswith("csv"))):
        sleep_file = file
        
    if ((sleep_data_file_prefix in file) and
        (file.endswith("csv"))):
        sleep_data_file = file
        
if ((sleep_file == None) or 
    (sleep_data_file == None)):
    print ("File not found")

print (sleep_file, sleep_data_file)

com.samsung.shealth.sleep.201809031104.csv com.samsung.shealth.sleep_data.201809031104.csv


In [3]:
## Read and process Sleep dataframe
pd_sleep_1 = pd.read_csv(data_dir+sleep_file, header=1)

cols_to_remove = ["com.samsung.health.sleep.custom", "extra_data", "com.samsung.health.sleep.comment",
                  "original_efficiency", "original_bed_time", "original_wake_up_time",
                  "com.samsung.health.sleep.pkg_name", "com.samsung.health.sleep.deviceuuid",
                  "com.samsung.health.sleep.create_time", "com.samsung.health.sleep.update_time"]

pd_sleep_1.drop(columns=cols_to_remove, inplace=True)

col_rename_dict = {"com.samsung.health.sleep.datauuid"   : "sleep_uuid",
                   "com.samsung.health.sleep.time_offset": "sleep_time_offset",
                   "com.samsung.health.sleep.end_time"   : "sleep_end_time",
                   "com.samsung.health.sleep.start_time" : "sleep_start_time"}

pd_sleep_1.rename(columns=col_rename_dict, inplace=True)

for col in pd_sleep_1.columns:
    if (pd_sleep_1[col].isnull().any()):
        print ("Null values in %s is %d" %(col, pd_sleep_1[col].isnull().sum()))
    else:
        print ("No null values in %s" %(col))

pd_sleep_1.shape

No null values in sleep_uuid
No null values in efficiency
Null values in has_sleep_data is 152
No null values in sleep_time_offset
No null values in sleep_end_time
Null values in quality is 202
No null values in sleep_start_time


(649, 7)

In [4]:
## Read and process sleep status dataframe
pd_sleep_2 = pd.read_csv(data_dir+sleep_data_file, header=1)

cols_to_remove = ["start_time", "pkg_name", "update_time", "create_time", "time_offset", "datauuid",
                  "comment", "deviceuuid", "json_version"]

pd_sleep_2.drop(columns=cols_to_remove, inplace=True)
print (pd_sleep_2.columns)
pd_sleep_2.shape

Index(['sleep_status', 'sleep_uuid'], dtype='object')


(498, 2)

In [5]:
## Merge the two dataframes - left join=sleep_uuid which is also the key
pd_sleep_data = pd.merge(left=pd_sleep_1, right=pd_sleep_2, how="left", on="sleep_uuid")
pd_sleep_data.shape

(649, 8)

In [6]:
pd_sleep_data.columns

Index(['sleep_uuid', 'efficiency', 'has_sleep_data', 'sleep_time_offset',
       'sleep_end_time', 'quality', 'sleep_start_time', 'sleep_status'],
      dtype='object')

In [7]:
def convert_ms_to_date_time_with_offset (df_series):
    
    ## Convert to UTC Date+Time
    df_series = pd.to_datetime(arg=df_series, unit="ms")
    
    ## Timezone conversion to local time
    df_series = df_series.dt.tz_localize("UTC").dt.tz_convert("Europe/Helsinki")
    
    return df_series

pd_sleep_data["sleep_start_time"] = convert_ms_to_date_time_with_offset(pd_sleep_data["sleep_start_time"])
pd_sleep_data["sleep_end_time"]   = convert_ms_to_date_time_with_offset(pd_sleep_data["sleep_end_time"])

In [8]:
pd_sleep_data.sort_values(ascending=True, by=["sleep_start_time"])

sleep_uuid  efficiency  has_sleep_data  \
375  91d4823e-90be-4c7a-8370-8961697a4c3d    0.000000             NaN   
273  6aeb5517-0af4-456f-a60c-e0b6c5565884    0.000000             NaN   
72   18a908df-30b6-4e54-9430-cbf1e5ddef0e    0.000000             NaN   
280  6e2c5424-ae9f-4797-8cb3-c786c4c8e502    0.000000             NaN   
522  d126088f-b435-45f7-b5e3-edc8ced42707    0.000000             NaN   
37   0de90c28-29f7-4edc-820a-48a93dcf3223    0.000000             NaN   
46   100c4f58-05a4-4245-9467-8036d4cbc3ae    0.000000             NaN   
39   0e37c478-25ce-4b8a-ad6f-934509ee64d2    0.000000             NaN   
68   168d63f0-5e26-40b5-8233-7bb0eef285b9    0.000000             NaN   
299  750df087-2abe-4bfc-b9e5-c77ff8b6e493    0.000000             NaN   
138  2f92a15f-44f9-419c-82ed-88b470466ae8    0.000000             NaN   
432  a6cda277-e886-4a07-9615-a889f23aebde    0.000000             NaN   
430  a68ee47f-984e-4111-abe7-12205739f3f9    0.000000             NaN   
102  2338c2b2-d492-41da-b6bc-ec040bf39587    0.000000             NaN   
80   1bbd15ed-6ed5-4099-8c4b-a2806a59f61e    0.000000             NaN   
504  c90930ca-773f-4b03-89b2-d391fe5c7c23    0.000000             NaN   
492  c4f6bec5-c3db-4f22-bba4-4f23516398ee    0.000000             NaN   
361  8db3d413-6f64-4c55-9277-43d38197bd6e    0.000000             NaN   
572  e5c24679-39c5-48a1-82d0-b05cda76392d    0.000000             NaN   
314  798b7d8e-e25d-49bb-aabf-c97048595f24    0.000000             NaN   
7    0229aa98-835a-4cf5-9150-b57a965a4c0d    0.000000             NaN   
164  393754c4-f482-4596-8804-b45d4fcf65c4    0.000000             NaN   
111  2661b891-4462-41ad-b0c6-c710a4135157    0.000000             NaN   
630  f9043dd9-0cfb-4d71-9b13-dd5f58f6693a    0.000000             NaN   
303  768f7612-9d54-4509-9f1d-cc0a109136f8    0.000000             NaN   
376  920f024f-e1b1-4a22-8e85-434042a5d13d    0.000000             NaN   
125  2b3e1b8d-c4ef-4a54-9709-431c190c1092    0.000000             NaN   
577  e6a06f2a-170b-46fe-b4bd-1f610259d2c4    0.000000             NaN   
290  721df36c-69f6-49e9-9df8-d6bd39359760    0.000000             NaN   
116  27f842bf-9230-456e-8e7a-06521c5a6ffc    0.000000             NaN   
..                                    ...         ...             ...   
106  24d9c3a8-28a2-e97c-419f-cf2f2e2315f5   93.195880             1.0   
185  40fd46e5-57d1-c771-a269-9b9c73ae85f7   90.126050             1.0   
83   1d3fdc9e-2913-a1db-6683-5ea1335fff49   91.842100             1.0   
552  d9e77341-ca94-fbc4-48ee-ac1605e6cb62   91.846530             1.0   
399  983288ea-24fd-d4ad-5569-a4e5c8f47c02   90.995255             1.0   
23   0a17afce-5106-d70b-3b08-0ea78858d648   92.105260             1.0   
393  968c3958-203c-c1f0-8793-e9741b10be85   93.621864             1.0   
543  d6e1ac10-366e-e91e-419a-1b82e1db0686   92.098090             1.0   
292  728b8028-f3d1-948c-b9b9-8ba77f6e0374   88.602150             1.0   
424  a4f9fecf-4c46-2879-12f1-595ac9c0ba57   83.076930             1.0   
294  7364986c-0794-997e-f8b4-b7237c8265c6   87.623764             1.0   
176  3b85161d-0d6f-1e86-e92b-e4df59bb1dbc   87.500000             1.0   
309  786efe83-8bf8-77e9-81e8-960c0947d407   92.694060             1.0   
22   09de09c4-ccc1-abef-b621-2e0878ba05c9   86.842110             1.0   
505  c920b902-27c7-cad1-70e3-7d1d7fa04ac9   91.448940             1.0   
369  90694f87-8273-e6c0-fbbb-498a712e95aa   92.488266             1.0   
618  f4a273c4-aa3d-e821-fd6b-58b2c172058e   93.954660             1.0   
502  c8336779-15ac-03b9-788e-b3741637efed   90.772540             1.0   
30   0c121b16-6cfd-c6f8-0e78-1fd5da7029db   89.444440             1.0   
153  3556a636-546e-6fc6-7484-32164737c71b   86.274510             1.0   
343  856314d8-bb43-0cbd-9a3f-b50256b6fc64   92.195120             1.0   
401  99e4a3cf-be9e-5ddd-4748-59e64c2d8e92   93.078760             1.0   
636  fc23a06e-6b21-7aef-75f0-394ee6eb874c   93.067230             1.0   
126  2c0fafd1-6e

In [9]:
sleep_json_dir = json_path + sleep_data_file_prefix + "/"

pd_sleep_data["sleep_status"] = pd_sleep_data["sleep_status"].fillna("None")

sleep_stat_dur = []
for item, row in pd_sleep_data.iterrows():
    json_file = row["sleep_status"]
    stuff = None

    if (json_file != "None"):
        json_abs_file = sleep_json_dir + json_file
        
        with open(json_abs_file) as f:
            json_data = json.load(f)
        
        index = 0
        json_data = sorted(json_data, key=lambda x: x["start_time"])

        while (index < len(json_data)):
            json_item = json_data[index]
            stat = json_item["status"]
            
            if (index+1 < len(json_data)):
                json_next_item = json_data[index+1]
                dur = json_next_item["start_time"] - json_item["start_time"]
            else:
                dur = 0

            found = False
            if (stuff != None):
                for subitem in stuff:
                    if (stat == subitem["stat"]):
                        subitem["dur"] += dur
                        found = True
                        break
                        
                if (not found):
                    stuff.append({"stat": stat, "dur": dur})
            else:
                stuff = [{"stat": stat, "dur": dur}]
            index += 1
    sleep_stat_dur.append(stuff)

In [10]:
## Col name list
col_list = []
for item in sleep_stat_dur:
    if (item != None):
        for subitem in item:
            if subitem["stat"] not in col_list: 
                col_list.append(subitem["stat"])

col_list = [("stat_"+str(x)) for x in sorted(col_list)]
col_list

['stat_0',
 'stat_10',
 'stat_20',
 'stat_30',
 'stat_40',
 'stat_50',
 'stat_60',
 'stat_70',
 'stat_80',
 'stat_90',
 'stat_100']

In [11]:
sleep_stat_dur

[None,
 [{'stat': 60, 'dur': 600000},
  {'stat': 30, 'dur': 600000},
  {'stat': 100, 'dur': 1200000},
  {'stat': 90, 'dur': 1200000},
  {'stat': 20, 'dur': 0}],
 [{'stat': 70, 'dur': 1200000},
  {'stat': 100, 'dur': 8400000},
  {'stat': 90, 'dur': 3600000},
  {'stat': 80, 'dur': 3000000},
  {'stat': 40, 'dur': 0}],
 [{'stat': 70, 'dur': 2400000},
  {'stat': 100, 'dur': 10200000},
  {'stat': 90, 'dur': 5400000},
  {'stat': 80, 'dur': 1200000},
  {'stat': 50, 'dur': 0}],
 None,
 [{'stat': 60, 'dur': 600000},
  {'stat': 90, 'dur': 6600000},
  {'stat': 100, 'dur': 6600000},
  {'stat': 80, 'dur': 3600000},
  {'stat': 70, 'dur': 600000},
  {'stat': 20, 'dur': 0}],
 [{'stat': 60, 'dur': 1200000},
  {'stat': 90, 'dur': 5400000},
  {'stat': 100, 'dur': 13200000},
  {'stat': 70, 'dur': 3000000},
  {'stat': 80, 'dur': 1200000},
  {'stat': 50, 'dur': 0}],
 None,
 [{'stat': 60, 'dur': 1200000},
  {'stat': 100, 'dur': 18000000},
  {'stat': 90, 'dur': 5400000},
  {'stat': 80, 'dur': 1800000},
  {'sta

In [12]:
stat_col_names = []
for n in range(0, 101, 10):
    col_name = "stat_" + str(n)
    stat_col_names.append(col_name)
    pd_sleep_data.insert(loc=len(pd_sleep_data.columns), column=col_name, value=0.0)

print (pd_sleep_data.columns)

Index(['sleep_uuid', 'efficiency', 'has_sleep_data', 'sleep_time_offset',
       'sleep_end_time', 'quality', 'sleep_start_time', 'sleep_status',
       'stat_0', 'stat_10', 'stat_20', 'stat_30', 'stat_40', 'stat_50',
       'stat_60', 'stat_70', 'stat_80', 'stat_90', 'stat_100'],
      dtype='object')


In [13]:
for ind, row in pd_sleep_data.iterrows():
    stat_dict = sleep_stat_dur[row.name]
    if (stat_dict != None):
        for j in stat_dict:
            col_name = "stat_" + str(j["stat"])
            pd_sleep_data.loc[ind, col_name] = j["dur"]

In [14]:
col_list = stat_col_names
col_list.append("efficiency")
col_list.append("sleep_start_time")
col_list.append("sleep_end_time")

In [15]:
pd_temp = pd_sleep_data[col_list].copy()

for ind, row in pd_temp.iterrows():
    for col in pd_temp.columns:
        if ((col != "efficiency") and 
            (col != "sleep_start_time") and 
            (col != "sleep_end_time") and
            (row[col] > 0)):
            pd_temp.loc[ind, col] = row[col] / (1000*60)

In [16]:
pd_temp.sort_values(ascending=False, by="sleep_start_time")

stat_0  stat_10  stat_20  stat_30  stat_40  stat_50  stat_60  stat_70  \
52      0.0      0.0      0.0      0.0      0.0     10.0     50.0     10.0   
195     0.0      0.0      0.0      0.0      0.0      0.0     20.0     30.0   
421     0.0      0.0      0.0      0.0      0.0      0.0     10.0     20.0   
42      0.0      0.0      0.0      0.0      0.0     10.0     10.0      0.0   
648     0.0      0.0      0.0      0.0      0.0     10.0      0.0     10.0   
139     0.0      0.0      0.0      0.0      0.0      0.0      0.0     10.0   
126     0.0      0.0      0.0      0.0      0.0     10.0      0.0     30.0   
636     0.0      0.0      0.0      0.0      0.0     10.0     10.0      0.0   
401     0.0      0.0      0.0      0.0      0.0     10.0      0.0     20.0   
343     0.0      0.0      0.0      0.0      0.0      0.0      0.0     20.0   
153     0.0      0.0      0.0      0.0      0.0     10.0     40.0     40.0   
30      0.0      0.0      0.0      0.0      0.0     20.0     30.0     20.0   
502     0.0      0.0      0.0     10.0      0.0     10.0     20.0     30.0   
618     0.0      0.0      0.0      0.0      0.0      0.0     10.0     20.0   
369     0.0      0.0      0.0      0.0      0.0      0.0      0.0     50.0   
505     0.0      0.0      0.0     10.0      0.0      0.0      0.0     30.0   
22      0.0      0.0      0.0      0.0      0.0      0.0     10.0      0.0   
309     0.0      0.0      0.0      0.0      0.0      0.0      0.0     20.0   
176     0.0      0.0      0.0      0.0      0.0      0.0     10.0      0.0   
294     0.0      0.0      0.0      0.0     10.0      0.0      0.0     30.0   
424     0.0      0.0      0.0      0.0     10.0      0.0      0.0      0.0   
292     0.0      0.0      0.0      0.0     10.0      0.0     20.0     60.0   
543     0.0      0.0      0.0      0.0      0.0      0.0      0.0     50.0   
393     0.0      0.0      0.0      0.0      0.0      0.0      0.0     20.0   
23      0.0      0.0      0.0      0.0     10.0      0.0     10.0     10.0   
399     0.0      0.0      0.0      0.0      0.0      0.0     20.0     20.0   
552     0.0      0.0      0.0      0.0      0.0      0.0      0.0     50.0   
83      0.0      0.0      0.0      0.0      0.0     10.0      0.0     20.0   
185     0.0      0.0      0.0      0.0      0.0      0.0     30.0     30.0   
106     0.0      0.0      0.0      0.0     10.0     20.0      0.0     10.0   
..      ...      ...      ...      ...      ...      ...      ...      ...   
116     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
290     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
577     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
125     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
376     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
303     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
630     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
111     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
164     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
7       0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
314     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
572     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
361     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
492     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
504     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
80      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
102     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
430     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
432     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
138     0.0      0.0      0

In [17]:
## stat_100 = Motionless_sleep
## stat_90 - stat_70 = Light sleep
## stat_60 - stat_00 = Restless

pd_sleep_data.rename(columns={"stat_100": "motionless_sleep_duration"}, inplace=True)
pd_sleep_data.insert(loc=len(pd_sleep_data.columns), column="light_sleep_duration", value=0)
pd_sleep_data.insert(loc=len(pd_sleep_data.columns), column="restless_duration", value=0)

for ind, row in pd_sleep_data.iterrows():
    pd_sleep_data.loc[ind, "light_sleep_duration"] = row["stat_90"] + row["stat_80"] + row["stat_70"]
    pd_sleep_data.loc[ind, "restless_duration"]    = row["stat_60"] + row["stat_50"] + row["stat_40"] + row["stat_30"] + row["stat_20"] + row["stat_10"]

pd_sleep_data.drop(["stat_90", "stat_80", "stat_70", "stat_60", "stat_50", "stat_40", "stat_30", "stat_20",
                    "stat_10", "stat_0"], axis=1, inplace=True)

In [18]:
pd_sleep_data.to_pickle("sleep_dataframe.pkl")